In [ ]:
# Importa a biblioteca PyGithub para interagir com a API do GitHub
from github import Github
import pandas as pd  # Importa a biblioteca pandas para manipulação de dados
import time  # Importa a biblioteca time para gerenciar temporizações
import os  # Importa a biblioteca os para interagir com o sistema operacional

# Lista de tokens do GitHub
tokens = [
    "SEU-TOKEN",  # Substitua por seu token pessoal do GitHub
    "SEU-TOKEN",  # Adicione mais tokens se necessário
    "SEU-TOKEN"   # Para contornar limites de taxa da API
]

# Variável para manter o índice do token atual
current_token_index = 0  # Inicia com o primeiro token

# Função para alternar o token


def get_next_token():
    global current_token_index  # Declara que estamos usando a variável global
    # Avança para o próximo token, retornando ao início se necessário
    current_token_index = (current_token_index + 1) % len(tokens)
    return tokens[current_token_index]  # Retorna o token atual


# Autenticação na API do GitHub com o primeiro token
# Cria uma instância do Github usando o token atual
g = Github(tokens[current_token_index])

# Conjunto de labels relacionados à experiência do usuário (UX)
ux_labels = {
    "Customer Experience", "CX", "User Experience",
    "Usability", "Ease of Use", "Usefulness",
    "User Interaction", "Human-Computer Interaction", "HCI", "Interaction Design",
    "User-Centered Design", "UCD", "Human-Centered Design", "HCD",
    "Interactive Experience", "Interactive Design", "User Engagement",
    "Digital Experience", "Online Experience", "Virtual Experience",
    "User Interface", "UI", "Interface Design", "UI Design",
    "Navigability", "Navigation", "User Navigation",
    "UX", "User Experience", "UX Design"
}

# Função para criar diretórios para armazenar os dados e imagens


def create_directories():
    # Verifica se o diretório '../Database' existe; se não, cria
    if not os.path.exists('../Database'):
        os.makedirs('../Database')
    # Verifica se o diretório '../Image' existe; se não, cria
    if not os.path.exists('../Image'):
        os.makedirs('../Image')


# Chama a função para criar os diretórios necessários
create_directories()

# Função para buscar repositórios relacionados a UX


def search_ux_repositories():
    repos_data = []  # Lista para armazenar os dados dos repositórios encontrados
    global g  # Utiliza a variável global para a instância do Github

    # Itera sobre cada label relacionado à experiência do usuário
    for label in ux_labels:
        # Define a consulta de busca para o label atual
        search_query = f"{label} in:description"
        # Imprime o label que está sendo buscado
        print(f"Buscando repositórios para: {label}")

        attempt = 0  # Contador de tentativas para mudar de token

        # Tentativas limitadas ao número de tokens disponíveis
        while attempt < len(tokens):
            try:
                # Busca repositórios no GitHub usando a consulta
                repos = g.search_repositories(
                    query=search_query, sort="stars", order="desc")  # Ordena por estrelas

                # Itera sobre os repositórios encontrados
                for repo in repos:
                    # Cria um dicionário com dados relevantes do repositório
                    repo_data = {
                        "name": repo.full_name,  # Nome completo do repositório
                        "description": repo.description,  # Descrição do repositório
                        "stars": repo.stargazers_count,  # Contagem de estrelas
                        "language": repo.language,  # Linguagem principal do repositório
                        "forks": repo.forks_count,  # Contagem de forks
                        "url": repo.html_url,  # URL do repositório
                        "created_at": repo.created_at,  # Data de criação
                        "updated_at": repo.updated_at,  # Data da última atualização
                        "open_issues": repo.open_issues_count,  # Contagem de issues abertas
                        "default_branch": repo.default_branch,  # Branch padrão
                        "contributors_count": repo.get_contributors().totalCount,  # Contagem de colaboradores
                        "commits_count": repo.get_commits().totalCount,  # Contagem de commits
                        "topics": repo.get_topics(),  # Tópicos do repositório
                        "watchers": repo.watchers_count,  # Contagem de watchers
                        "size": repo.size,  # Tamanho do repositório
                        "has_wiki": repo.has_wiki,  # Indica se o repositório tem uma wiki
                        "has_projects": repo.has_projects,  # Indica se o repositório tem projetos
                        "has_issues": repo.has_issues,  # Indica se o repositório tem issues
                        "has_pages": repo.has_pages,  # Indica se o repositório tem páginas GitHub
                        "releases_count": repo.get_releases().totalCount  # Contagem de releases
                    }

                    # Adiciona os dados do repositório à lista
                    repos_data.append(repo_data)

                break  # Se a busca for bem-sucedida, sai do loop de tentativas

            except Exception as e:  # Captura qualquer exceção que ocorrer durante a busca
                # Imprime o erro encontrado
                print(f"Erro ao buscar repositórios para o termo {label}: {e}")
                # Informa que irá tentar outro token
                print("Tentando outro token...")
                g = Github(get_next_token())  # Alterna para o próximo token
                attempt += 1  # Incrementa o contador de tentativas
                time.sleep(1)  # Pausa para evitar sobrecarregar a API
                continue  # Continua com a próxima tentativa

        time.sleep(1)  # Pausa adicional para evitar limite de taxa da API

    # Remove duplicatas dos dados coletados e salva em um arquivo CSV
    df = pd.DataFrame(repos_data).drop_duplicates(subset='name')
    df.to_csv('../Database/Resultados-Busca/repositorios_ux.csv',
              index=False, sep=',', decimal=',')  # Salva os dados em um arquivo CSV
    # Confirmação de conclusão
    print("Coleta de repositórios concluída e salva no arquivo '../Database/repositorios_ux.csv'.")


# Chama a função para buscar repositórios relacionados a UX
search_ux_repositories()